In [4]:
import partitura as pt
import numpy as np
import copy
import os

In [5]:
def reflect(chord, point=0):
    """Reflect a chord around a point."""
    base = chord[0]
    new_base = ((point + 7) - base) % 12
    if len(chord) > 1:
        rel_from_base = [(n - base) for n in chord[1:]]
        inv_rel = [(-rel) % 12 for rel in rel_from_base]
        neg_chord = [new_base] + list(map(lambda x: (x + new_base) % 12, inv_rel))
    else:
        neg_chord = [new_base]
    return neg_chord

In [14]:
def negative_harmony(old_part):
    """Create Negative Harmony in the JacobColierian way."""
    part = copy.copy(old_part)
    na = part.note_array()
    has_part = True
    for onset in np.unique(na["onset_beat"]):
        plist = na[np.where(na["onset_beat"] == onset)]["pitch"]
        pm = list(map(lambda y: (y % 12, int(y / 12)), plist))
        pc, mapping = zip(*pm)
        new_pc = reflect(chord=pc, point=0)
        new_pitches = [a + b * 12 for a, b in zip(new_pc, mapping)]
        for i, index in enumerate(np.where(na["onset_beat"] == onset)[0]):
            na["pitch"][index] = new_pitches[i]

    # Overwite the part with new pitches
    for note in part.notes:
        n = na[na["id"] == note.id]
        mpitch = n["pitch"].item()
        note_spelling = pt.utils.music.midi_pitch_to_pitch_spelling(mpitch)
        note.step, note.alter, note.octave = note_spelling
    return part

In [15]:
# Replace by your piece path
piece_path = os.path.join(os.path.dirname(os.getcwd()), "assets", "Exercise_01.musicxml")
part = pt.load_score(piece_path)[0]

In [16]:
new_part = negative_harmony(part)
pt.save_musicxml(new_part, os.path.join(os.getcwd(), "negative_part.musicxml"))